# Inhaltsverzechnis
- DataArray und Dataset
- Dimensionen und Koordinaten
- Indizierung
- `xarray.where` Array Auswahl nach beliebigen Kriterien

# Einführung zu xarray

xarray ist ein Pythonpaket für die Verarbeitung von Daten im Netcdf Format. Xarray ist sehr hilfreich für die Verarbeitung von netcdf files mit meteorologischen Daten. 

## Abhängigkeiten
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

xarray braucht mindestestens 
- `numpy`
- `pandas`
- `packaging`.

Allerdings gibt es viele hilfreiche Funktionalitäten, die nur nutzbar sind, wenn weitere Abhängigkeiten erfüllt sind. Besonders nützlich sind (Auswahl!):
- `dask`: lazy loading von großen Dateien. Operationen werden parallel und erst auf Aufforderung ausgeführt.
- `netCDF4`
- `scipy`
- `cftime`: für die Verarbeitung von Datumsformaten
- `matplotlib`, `cartopy`


## Häufige Probleme
### numpy version
Relativ häufig gibt es Probleme beim Lesen von Dateien, wenn eine neue `xarray` version auf eine ältere `numpy` Version trifft. Da hilft nur, sich eine environment anzulegen und hier ein neueres numpy zu installieren. 

#### netcdf Bibliothek
Es kann nervig werden, wenn die C-Bibliotheken inklusive devel Pakete auf SuSe nicht installiert sind. 

### Differenzierung zwischen Dimensionen und Koordinaten
xarray unterscheidet relativ strikt zwischen sogenannten Dimensionen oder Koordinaten. Je nachdem mit welchem Werkzeug man eine netcdf Datei erstellt hat, kann es sein, dass man den Datensatz erst umformatieren muss, bevor die praktischen xarray Funktionalitäten ansteuerbar sind. 

Eine Dimension hat einen Namen und gibt an wieviele Einträge es in dieser Dimension gibt, z.B. `ncells = 600000`. 

Eine Koordinate ist z.B. die geographische Breite. Sie hat eine oder mehrere Dimensionen. 

Viele Plotroutinen von xarray gehen nach Koordinaten oder Dimensionen. 

## inplace oder nicht?
"Inplace" bedeutet, dass eine Funktion, die zu einem Objekt gehört, dieses dauerhaft selbst verändert. Viele Funktionen in xarray sind NICHT inplace. D.h. wenn man irgendwas mit einem dataset oder dataarray tut, muss man den output in eine neue Variable schreiben, oder die vorherige überschreiben. 

In [ ]:
import numpy as np
import datetime as dt
import xarray
import matplotlib.pyplot as plt

# xarray Basics

## Dataset und Dataarrays erstellen und Auswählen

-> simples Beispiel, normalerweise nützlich, wenn man Daten irgendwoher schon hat und die mit xarray weiterverarbeiten will

Quelle: https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html

In [ ]:
dataArray = xarray.DataArray(np.random.randn(2, 3), dims=("x", "y"), coords={"x": [10, 20]})

In [ ]:
dataArray

In [ ]:
dataSet = xarray.Dataset(dict(foo=dataArray, bar=("x", [1, 2]), baz=np.pi))

In [ ]:
dataSet

In [ ]:
dataSet.foo

In [ ]:
dataSet['x'] # key syntax

In [ ]:
dataSet.foo.dims

In [ ]:
dataSet.foo.coords

In [ ]:
# add some new coordinates with dimension x
# Caution: assign_coords does not work on the array itself (inplace), you have to save the output!
dataSet = dataSet.assign_coords(dict(**dataSet['foo'].coords, lat=('x',np.array([50,70]))))
dataSet

In [ ]:
dataSet.drop_vars(['baz']) # remove baz 

In [ ]:
dataSet.drop_vars(['foo','bar'])

In [ ]:
dataSet.data_vars

In [ ]:
type(dataSet['foo'])

In [ ]:
type(dataSet['foo'].values)

## Indizieren

### isel: Selektion nach index, d.h. nach n-tem Eintrag einer Dimension

In [ ]:
dataSet.isel(dict(y=[1],x=[0]))

In [ ]:
dataSet.isel(dict(y=[1],x=0)) # drops x Dimension! 

In [ ]:
dataSet.isel(dict(y=[1],x=0)).drop_dims('y') # explicitely drop dimension y

### sel: Auswahl nach Koordinate

In [ ]:
dataSet.sel(x=[20])

### boolean array

In [ ]:
dataArray.shape

In [ ]:
msk = np.repeat(np.array([[True,False]]),3).reshape((dataArray.shape))
msk

In [ ]:
dataArray

In [ ]:
dataArray.values[msk] # numpy array index

### xarray.where

In [ ]:
xa_msk = xarray.DataArray(msk,coords=dataArray.coords)
xa_msk

In [ ]:
dataArray.where(xa_msk)

In [ ]:
dataSet.where(dataSet.foo>0)

In [ ]:
dataSet.foo>0

In [ ]:
xarray.where(dataSet.foo>0,dataSet.foo, np.inf) # 1st argument is a boolean xarray

In [ ]:
xarray.where(dataSet.foo>0,7, np.inf) 